# Imports

In [57]:
import numpy as np
import pandas as pd
import statsmodels as sm
import sklearn as skl
import math
import matplotlib as mpl

# Einlesen und aufbereiten

In [58]:
updated_cols = pd.read_excel("catan_data.xlsx").iloc[0].T
df = pd.read_excel("catan_data.xlsx")
renamedict = dict(zip(df.columns, list(updated_cols)))
df = df.rename(columns = renamedict).iloc[1:]

C:\Users\maxfu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\maxfu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [59]:
#numerisch machen
for col in df.columns:
    df[col] = pd.to_numeric(df[col],errors = "ignore")

In [60]:
#variable für direkt am hafen gebaut
df["port"] = 0
df.loc[(df["port_wood"]==1) | (df["port_clay"]==1) | (df["port_sheep"]==1) | 
       (df["port_grain"]==1 )| (df["port_ore"]==1) | (df["port_gen"]==1),"hafen"] = 1

#variable für in hafennähe gebaut
df["prox_port"] = 0
df.loc[(df["prox_port_wood"]==1) | (df["prox_port_clay"]==1) | (df["prox_port_sheep"]==1) | 
       (df["prox_port_grain"]==1 )| (df["prox_port_ore"]==1) | (df["prox_port_gen"]==1),"hafen"] = 1

#binärvariablen für spieler
for player in df["player"].unique():
    df[player] = 0
    df.loc[df["player"]==player,player] = 1
    
#shares berechnen
resources = []
for colname in [col for col in df if "t_sum" in col]:
    resources.append(colname[6:])
for resource in resources:
    df["share_"+resource] = df["p_sum_"+resource] / df["t_sum_"+resource]

# Auswertungen

In [20]:
[col for col in df]

['season',
 'Session',
 'game',
 'loc',
 'player',
 'score',
 'special',
 'place',
 'position',
 'desert',
 'sum',
 'count_num',
 'count_res',
 'p_sum_wood',
 'p_sum_clay',
 'p_sum_sheep',
 'p_sum_grain',
 'p_sum_ore',
 'p_sum_paper',
 'p_sum_coin',
 'p_sum_fabric',
 'port_wood',
 'port_clay',
 'port_sheep',
 'port_grain',
 'port_ore',
 'port_gen',
 'prox_port_wood',
 'prox_port_clay',
 'prox_port_sheep',
 'prox_port_grain',
 'prox_port_ore',
 'prox_port_gen',
 't_sum_wood',
 't_sum_clay',
 't_sum_sheep',
 't_sum_grain',
 't_sum_ore',
 't_sum_paper',
 't_sum_coin',
 't_sum_fabric',
 'hafen',
 'PF',
 'MF',
 'JHC',
 'prox_port',
 'port',
 'share_wood',
 'share_clay',
 'share_sheep',
 'share_grain',
 'share_ore',
 'share_paper',
 'share_coin',
 'share_fabric']

In [80]:
shares = {}
for player in df["player"].unique():
    x = []
    for resource in resources:
        t = df.loc[df[player]==1]["share_"+resource].describe()[["mean","std"]]
        t = pd.DataFrame(t).T.rename(columns = {"mean":"mean_"+player,"std":"std_"+player})
        x.append(t)
    shares[player] = pd.concat(x)

shares = pd.concat(shares.values(),axis = 1)

In [84]:
shares.to_excel("xxv001 - resource shares by player.xlsx")